In [ ]:
!pip install cpi

In [1]:
import cpi
import pandas as pd

In [2]:
# Inflation table
def inflation_table(base_year):
    cpi.update()
    series_df = cpi.series.get(area="U.S. city average").to_dataframe()
    inflation_df = (series_df[series_df.year >= 2008]
           .pivot_table(index='year', values='value', aggfunc='mean')
           .reset_index()
          )
    denominator = inflation_df.value.loc[inflation_df.year==base_year].iloc[0]

    inflation_df = inflation_df.assign(
        inflation = inflation_df.value.divide(denominator)
    )
    
    return inflation_df

In [3]:
cpi_table = inflation_table(2021)
cpi_table

,year,value,inflation
0,2008,215.302538,0.794563
1,2009,214.537000,0.791738
2,2010,218.055538,0.804723
3,2011,224.939154,0.830126
4,2012,229.593923,0.847305
5,2013,232.957077,0.859716
6,2014,236.736154,0.873663
7,2015,237.017000,0.874699
8,2016,240.007154,0.885734
9,2017,245.119615,0.904601


In [7]:
df = pd.read_parquet("dla_df.parquet")

keep_cols = [
    "primary_agency_name", "project_no", "prefix", "prepared_y",
]

orig = ["total_requested", 
        #"fed_requested", 
        #"ac_requested",  
]

adj = ["adjusted_total_requested", 
       #"adjusted_fed_requested", 
       #"adjusted_ac_requested"
]

df = df[keep_cols + orig + adj]

# Pick ones that have values so that we can check
df = df[(df.total_requested > 0) & (df.total_requested.notna())]
df[orig] = df[orig].astype(int)
#df = df[(df.fed_requested > 0) & (df.ac_requested > 0)]

In [8]:
df2 = pd.merge(df, 
         cpi_table[["year", "inflation"]],
         left_on = "prepared_y",
         right_on = "year",
         how = "left",
         validate = "m:1",
        )

In [10]:
for c in orig:
    new_col = f"{c}2"
    df2[new_col] = df2.apply(lambda x: x[c] * x.inflation, axis=1)

In [11]:
for c in orig:
    new_col = f"compare_{c}"
    natalie_col = f"adjusted_{c}"
    tiff_col = f"{c}2"
    df2[new_col] = df2.apply(lambda x: 1 if x[natalie_col] == x[tiff_col]
                             else 0, axis=1)

In [15]:
# If everything is in 2021 dollars
# unadjusted and adjusted amts should be equal
df2[df2.prepared_y ==2021]

,primary_agency_name,project_no,prefix,prepared_y,total_requested,adjusted_total_requested,year,inflation,total_requested2,compare_total_requested
11068,Sacramento County,5924(252),CML,2021.0,405260,4.052603e+05,2021.0,1.0,405260.0,0
11075,Folsom,5288(039),BRLS,2021.0,15456,1.545698e+04,2021.0,1.0,15456.0,0
11082,El Cerrito,5239(029),CML,2021.0,965000,9.650008e+05,2021.0,1.0,965000.0,0
11085,San Jose,5005(129),CML,2021.0,1049114,1.049116e+06,2021.0,1.0,1049114.0,0
11086,Marin County,5927(114),HSIPL,2021.0,711540,7.115407e+05,2021.0,1.0,711540.0,0
...,...,...,...,...,...,...,...,...,...,...
12332,Marin County Transit District,6343(006),FTASTPL,2021.0,1120000,1.120001e+06,2021.0,1.0,1120000.0,0
12333,Santa Clara Valley Transportation Authority,6264(091),FTASTPL,2021.0,1987000,1.987002e+06,2021.0,1.0,1987000.0,0
12334,Alameda - Contra Costa Transit District,6002(030),FTASTPL,2021.0,1254000,1.254001e+06,2021.0,1.0,1254000.0,0
12342,San Francisco Bay Area Rapid Transit District,6000(069),FTACRRSA,2021.0,4335000,4.335004e+06,2021.0,1.0,4335000.0,0


In [20]:
for c in ["total_requested", "adjusted_total_requested", "total_requested2"]:
    num = df2[df2.prepared_y ==2021][c].iloc[0]
    print(num)

405260
405260.3451364289
405260.0


In [22]:
# If everything is in 2021 dollars, years prior to 2021 have multiplier < 1
# So adjusted values should be smaller than unadjusted
df2[df2.prepared_y == 2020][orig + adj]

,total_requested,adjusted_total_requested
9881,680000,7.119461e+05
9889,759180,7.948460e+05
9890,0,1.361073e-01
9893,115000,1.204027e+05
9897,367000,3.842415e+05
...,...,...
12146,189637,1.985461e+05
12162,2694349,2.820929e+06
12163,1553769,1.626765e+06
12184,77118,8.074097e+04


In [23]:
df2[df2.prepared_y == 2020][orig + ["total_requested2"]]

,total_requested,total_requested2
9881,680000,6.494879e+05
9889,759180,7.251150e+05
9890,0,0.000000e+00
9893,115000,1.098399e+05
9897,367000,3.505324e+05
...,...,...
12146,189637,1.811278e+05
12162,2694349,2.573452e+06
12163,1553769,1.484050e+06
12184,77118,7.365766e+04
